In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np
import re

import random
from random import sample

from tqdm import tqdm

/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [3]:
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [5]:
model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
                                                 torch_dtype=torch.float16,
                                                 low_cpu_mem_usage=True).cuda()

Loading checkpoint shards: 100%|██████████| 13/13 [00:00<00:00, 14.91it/s]


In [20]:
print(model.config.max_position_embeddings)
print(model.config.max_length)
model.config.max_length = 2048
print(model.config.max_length)
model.config.pad_token_id = 0

2048
2048
2048


In [3]:
# Data
df_train = pd.read_json('dataset/total_train_dataset.json')
df_val = pd.read_json('dataset/total_val_dataset.json')

In [4]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [5]:
df_train.head()

,index,info.essay_id,info.essay_prompt,paragraph,score.essay_scoreT_avg,student.student_grade,paragraph2
0,0,ESSAY_72480,"여러분은 저작권에 대해 들어보셨나요? 저작권이란, 저작권을 가진 소유자가 자신의 ...",저는 카피레프트 운동을 실시하는 것에 대해 찬성하는 입장입니다. 카피레프트로 인해 ...,22.333334,중등_2학년,저는 카피레프트 운동을 실시하는 것에 대해 찬성하는 입장입니다.#@문장구분# 카피레...
1,0,ESSAY_52262,반려동물을 키우는 가정이 급속도로 늘어나고 있습니다. 그에 비례하여 반려동물 및 ...,나는 동물사육을 제한하는 것은 옳지 않다고 생각한다.\n 분명히 제한 규제나 다른 ...,28.600000,중등_3학년,나는 동물사육을 제한하는 것은 옳지 않다고 생각한다.#@문장구분#\n 분명히 제한 ...
2,0,ESSAY_54605,반려동물을 키우는 가정이 급속도로 늘어나고 있습니다. 그에 비례하여 반려동물 및 ...,"지금 반려동물에 의해 일어나는 피해사례가 많아지고 있어, 사육에 대한 제도를 강화하...",28.466667,중등_3학년,"지금 반려동물에 의해 일어나는 피해사례가 많아지고 있어, 사육에 대한 제도를 강화하..."
3,0,ESSAY_51646,과학 발전을 통해 다양한 에너지들이 만들어지고 일상생활에 사용되고 있습니다. 대표...,저는 생활의 편리함을 뒤로하고 에너지 절약을 먼저 하자는 의견입니다 환경은 미래에 ...,28.111109,초등_6학년,저는 생활의 편리함을 뒤로하고 에너지 절약을 먼저 하자는 의견입니다 #@문장구분#환...
4,0,ESSAY_68296,반려동물을 키우는 가정이 급속도로 늘어나고 있습니다. 그에 비례하여 반려동물 및 ...,나는 동물을 키우는 것을 법을 반대하는 것이 좋겠다고 생각하고 있다. 왜냐하면 우리...,28.066666,중등_3학년,나는 동물을 키우는 것을 법을 반대하는 것이 좋겠다고 생각하고 있다. #@문장구분#...


In [31]:
# full dataset
train_data = []

for prompt, sent, label in tqdm(df_train[['info.essay_prompt', 'paragraph', 'score.essay_scoreT_avg']].values):
    # tokens = tokenizer(prompt + '\n' + sent).input_ids
    train_data.append((prompt, sent, label))
    
train_fewshot = sample(train_data, 10)

100%|██████████| 39591/39591 [00:00<00:00, 512125.49it/s]


In [59]:
# prompt

def build_prompt_text(sent):
    return "\n\n### 다음 답안의 수준을 0점부터 30점까지의 점수로 평가해주세요.\n" + sent + "\n점수: "

def clean_text(sent):
    # sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    sent_clean = sent
    return sent_clean

real_labels = []
pred_tokens = []

total_len = len(df_val[['info.essay_prompt', 'paragraph', 'score.essay_scoreT_avg']].values)
for i, (prompt, sent, label) in tqdm(enumerate(df_val[['info.essay_prompt', 'paragraph', 'score.essay_scoreT_avg']].values), total=total_len):
    prompt_text = ''
    
    for j in range(len(train_fewshot)):
        ex_prompt, ex_text, ex_label = train_fewshot[j]
        # cleaned_ex_text = clean_text('질문: ' + ex_prompt + '\n' + '답안: ' +  ex_text)
        cleaned_ex_text = clean_text('답안: ' +  ex_text)
        appended_prompt_ex_text = build_prompt_text(cleaned_ex_text)
        appended_prompt_ex_text += str(ex_label)
        prompt_text += appended_prompt_ex_text
    
    # ex_prompt, ex_text, ex_label = train_data[i]
    # cleaned_ex_text = clean_text(ex_prompt + '\n' + ex_text)
    # appended_prompt_ex_text = build_prompt_text(cleaned_ex_text)
    # appended_prompt_ex_text += '\n' + ex_label
    # promp_text += appended_prompt_ex_text
    
    # cleaned_sent = clean_text('질문: ' + prompt + '\n' + '답안: ' + sent)
    cleaned_sent = clean_text('답안: ' + sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)
    
    prompt_text += appended_prompt_sent
    
    tokens = tokenizer(prompt_text, return_tensors='pt',  max_length=2048, truncation=True, padding=True)
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    gen_tokens = model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()
    
    try:
        pred_tokens.append(float(pred))
    except ValueError:
        pred_tokens.append(pred)
    real_labels.append(float(label))

  1%|          | 31/5906 [00:16<51:35,  1.90it/s]


KeyboardInterrupt: 

In [56]:
print(prompt_text)



##### 다음 답안의 수준을 0점부터 30점까지의 점수로 평가해주세요.
답안: 위에 나온 가사의 곡은 중년의 가수 '김연자'님께서 만든 노래입니다. 중년의 가수분도 결혼은 선택이라고 생각하시는 모습을 보면 이미 대한민국의 사람들 대부분에게는 결혼은 선택이라는 생각을 가지고 있다고 볼 수가 있습니다. 사실 예전에는 결혼이 거의 필수라고 생각되었고, 출산 또한 4~5명씩 낳았을만큼 대를 이어가는 것에 대해서 큰 의미를 두었지만, 세상은 변했다고 생각합니다. 어떤 길도 좋고 나쁨을 단정지을수는 없고, 어떤 길을 선택할 것인지도, 개인의 자유입니다. 이만큼 개인의 자유를 보장해주는 것이 중요해진 사회에서 결혼을 필수라고 말하는 사람은 아마 고지식한 사람으로 생각될 수 있습니다. 또한 세상에는 여러 생각을 가진 사람이 있고, 그중에는 비혼주의를 가진 사람도 있습니다. 물론 저는 비혼주의가 아니라서 왜 그런 생각을 가지고 있는지는 알 수 없지만, 아마 자신의 삶을 결혼해서의 삶보다 더 중요시 여기기 때문이라고 생각합니다. 확실히 결혼을 하고 아이를 낳게 되면 OO엄마, OO아빠로 불리는 경우가 많습니다. 자신의 이름으로 자신의 삶을 살아가고 싶은 사람들이 비혼을 하거나 또는 결혼을 하더라도 출산을 하지 않으려고 하는 것이라고 생각합니다. 또한 아이를 낳아 키우려면 경제적인 비용도 많이 들기 때문에 출산까지는 많은 사람들이 하지 않으려고 합니다. 아이를 낳아 키우고 싶지만 경제적인 문제 때문에 꺼려하는 사람들도 있다고 생각합니다. 이런 여러 처지를 가진 사람들이 있기 때문에 결혼을 필수라고 말하기는 힘들 것이라고 생각합니다. 힘든 일을 억지로 하는 것은 그 사람에게 부담을 줄 수 있는 행위이며 온전히 본인의 삶이기 때문에 누구도 참견할 자격은 없습니다.
점수: 28.666666

##### 다음 답안의 수준을 0점부터 30점까지의 점수로 평가해주세요.
답안: 다문화 가정은 아버지 혹은 어머니가 외국 국적일 경우를 말합니다.    
국가 정책을 통해 다문화 가족에 대해

In [60]:
# eval
from scipy.stats import pearsonr
score = pearsonr(pred_tokens, real_labels)
print(score)

PearsonRResult(statistic=nan, pvalue=nan)


/opt/conda/envs/llm/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


# Full Finetuning

In [12]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [1]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [3]:
import pandas as pd
df = pd.read_json('train_v1.1b/KoAlpaca_v1.1a_textonly.json', lines=True)
df.head()
# print(len(df))
# print(df.iloc[0]['text'])

,text
0,### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n##...
1,### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 ...
2,### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 ...
3,### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 ...
4,"### 질문: '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것..."


In [6]:
df['text'][0]

'### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'

In [2]:
print(df.iloc[0]['text'])

### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/opt/conda/envs/llm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/llm2/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/envs/llm2/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/envs/llm2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/envs/llm2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/envs/llm2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAAAgQC67Y74y+iOqa54+F/HqDyOlGVdWnUlcs62znFMxJrREL57bu9ZnaEPflvXBSs09I4mSAbii49awsO1RdtZNjMJiLLMcObb9NWOqw2FHo0zCK7XYUd84tJ1qm73B8uLDGPpuo5laKCPEhl5bfqMG63Qb51vl1PlWDMcs4B7mXl7+w==')}
  warn(msg)
/opt/conda/envs/llm2/lib/pyt

In [ ]:
!nvidia-smi

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [3]:
# Data
import pandas as pd
df_train = pd.read_json('dataset/total_train_dataset.json')
df_val = pd.read_json('dataset/total_val_dataset.json')

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

NameError: name 'model' is not defined

In [90]:
def make_prompt(grade, essay_prompt, paragraph, score):
    result = '### 학년: ' + grade + '\n\n'
    result += '### 질문: ' + essay_prompt + '\n\n'
    result += '### 답안: ' + paragraph + '\n\n'
    result += '### 점수: ' + str(score)+ '<|endoftext|>'
    return result

In [91]:
make_prompt('고등3', '뭐냐?', '나다', '32')

'### 학년: 고등3\n\n### 질문: 뭐냐?\n\n### 답안: 나다\n\n### 점수: 32<|endoftext|>'

In [99]:
from datasets import Dataset
# full dataset
train_dataset = []
for grade, essay_prompt, paragraph, score in tqdm(df_train[['student.student_grade', 'info.essay_prompt', 'paragraph', 'score.essay_scoreT_avg']].values):
    train_dataset.append({'text':make_prompt(grade, essay_prompt, paragraph, score)})

# to huggingface dataset
train_dataset = Dataset.from_dict(pd.DataFrame(train_dataset))

100%|██████████| 39591/39591 [00:00<00:00, 127817.23it/s]


In [103]:
# tokenization
train_dataset = train_dataset.map(lambda samples: tokenizer(samples['text']), batched=True)

In [104]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=500,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


NameError: name 'str2optimizer8bit_blockwise' is not defined

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 답안: 노트북이 꼭 맥북이 아니더라도 스타벅스에 들어갈 수 있어야 한다. 왜냐하면 누구든지 자신이 마음에 드는 노트북을 가지고 다닐 권리가 있기 때문이다.", return_tensors='pt', return_token_type_ids=False))